In [1]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import random
import datetime
%load_ext tensorboard

In [2]:
from google.cloud import datastore
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = \
    "/home/alice/Downloads/HCL-customer-application-c05a56fa9ccc.json"


client = datastore.Client()
query = client.query(kind="pricedata")
x = []
y = []
for i in query.fetch():
    x.append([i['smolprice'], i['chonkprice'], i['absprice'],i["balance"]])
    y.append([i['coins']])


In [3]:
from tensorflow.keras import layers

model = tf.keras.Sequential()
# Adds a densely-connected layer with 64 units to the model:
model.add(layers.Dense(2, activation='relu',input_shape=(3,)))
# Add an output layer with 10 output units:
model.add(layers.Dense(1))

In [4]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='mse',       # mean squared error
              metrics=['mae'])  # mean absolute error

In [5]:
import numpy as np

In [6]:
data = np.asarray(x)
target = np.asarray(y)
print(data)
print(target)

[[   5  505  555 1204]]
[[360]]


In [7]:

def Normalize(data, mean_data =None, std_data =None):
    if not mean_data:
        mean_data = np.mean(data)
    if not std_data:
        std_data = np.std(data)
    norm_data = (data-mean_data)/std_data
    return norm_data, mean_data, std_data

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2)

X_train, xmean_data, xstd_data = Normalize(X_train)
X_test, _, _ = Normalize(X_test, xmean_data, xstd_data)

Y_train, ymean_data, ystd_data = Normalize(y_train)
Y_test,_,_ = Normalize(y_test, ymean_data, ystd_data)

ValueError: With n_samples=1, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [8]:
print(X_train)
print(Y_train)

NameError: name 'X_train' is not defined

In [116]:
print(X_test)
print(Y_test)

[[-0.86473387 -0.81225002  0.55233001]
 [ 1.62824888  0.05373346 -0.99594348]]
[[-1.95988019]
 [-0.93530856]]


In [117]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(X_train, Y_train, 
          validation_data=(X_test,Y_test), 
          batch_size=20, 
          epochs=25,
          verbose=1,
         callbacks=[tensorboard_callback])

Epoch 1/25
1/1 [==============================] - 0s 84ms/step - loss: 0.6064 - mae: 0.6758 - val_loss: 1.9406 - val_mae: 0.9992
Epoch 2/25
1/1 [==============================] - 0s 34ms/step - loss: 0.5870 - mae: 0.6669 - val_loss: 1.9616 - val_mae: 1.0183
Epoch 3/25
1/1 [==============================] - 0s 40ms/step - loss: 0.5690 - mae: 0.6581 - val_loss: 1.9835 - val_mae: 1.0375
Epoch 4/25
1/1 [==============================] - 0s 31ms/step - loss: 0.5523 - mae: 0.6493 - val_loss: 2.0064 - val_mae: 1.0568
Epoch 5/25
1/1 [==============================] - 0s 30ms/step - loss: 0.5370 - mae: 0.6406 - val_loss: 2.0302 - val_mae: 1.0761
Epoch 6/25
1/1 [==============================] - 0s 31ms/step - loss: 0.5231 - mae: 0.6318 - val_loss: 2.0550 - val_mae: 1.0950
Epoch 7/25
1/1 [==============================] - 0s 33ms/step - loss: 0.5104 - mae: 0.6231 - val_loss: 2.0805 - val_mae: 1.1134
Epoch 8/25
1/1 [==============================] - 0s 35ms/step - loss: 0.4987 - mae: 0.6144 - val

In [118]:
%tensorboard --logdir logs/fit


Reusing TensorBoard on port 6006 (pid 25935), started 0:59:26 ago. (Use '!kill 25935' to kill it.)

In [102]:
z = np.asarray([[5.0,5.0,5.0]])
dat = Normalize(z, xmean_data, xstd_data)[0]
print(model.predict(dat)[0][0]*ystd_data + ymean_data)

z = np.asarray([[300.0,40.0,30.0]])
dat = Normalize(z, xmean_data, xstd_data)[0]
print(model.predict(dat)[0][0]*ystd_data + ymean_data)

z = np.asarray([[400.0,200.0,30.0]])
dat = Normalize(z,xmean_data, xstd_data)[0]
print(model.predict(dat)[0][0]*ystd_data + ymean_data)


1374.3482228218795
1495.3198652902636
1063.785709711485


In [9]:
    data = []
    maxval = 0
    stepsize = 50
    maxvaldat = [[5, 5, 5]]
    for smol in range(5, 600, stepsize):
        for chonk in range(5, 600, stepsize):
            for abso in range(5, 600, stepsize):
                x = np.array([[smol, chonk, abso]])
                dat = Normalize(x, xmean_data, xstd_data)[0]
                pred = model.predict(dat)
                data.append([smol,chonk,abso,pred[0][0] *ystd_data + ymean_data])
                if(pred > maxval):
                    maxval = pred
                    maxvaldat = [[smol, chonk, abso]]


NameError: name 'xmean_data' is not defined

In [104]:
print(maxval)
print(maxvaldat)

[[1.9484779]]
[[555, 5, 5]]


In [105]:
def keyfunc(e):
    return e[3]

In [106]:
data.sort(reverse=True,key=keyfunc)

In [107]:
data

[[555, 5, 5, 1980.0358109257947],
 [505, 5, 5, 1924.9733611790762],
 [455, 5, 5, 1869.9109114323574],
 [555, 5, 55, 1832.100836221949],
 [405, 5, 5, 1814.8483335076367],
 [555, 55, 5, 1794.4220152855971],
 [505, 5, 55, 1777.0382582972284],
 [355, 5, 5, 1759.7858837609183],
 [505, 55, 5, 1739.3595014498774],
 [455, 5, 55, 1721.9758085505098],
 [305, 5, 5, 1704.7233058361976],
 [455, 55, 5, 1684.2969876141578],
 [555, 5, 105, 1684.1657333401013],
 [405, 5, 55, 1666.913230625789],
 [255, 5, 5, 1649.66092017848],
 [555, 55, 55, 1646.4869764927505],
 [405, 55, 5, 1629.234537867439],
 [505, 5, 105, 1629.1031554153806],
 [355, 5, 55, 1611.8507808790705],
 [555, 105, 5, 1608.8080914673974],
 [205, 5, 5, 1594.598278164758],
 [505, 55, 55, 1591.4244626570307],
 [355, 55, 5, 1574.1719599427186],
 [455, 5, 105, 1574.040705668662],
 [305, 5, 55, 1556.7882029543498],
 [505, 105, 5, 1553.745641720679],
 [155, 5, 5, 1539.5358284180395],
 [455, 55, 55, 1536.36188473231],
 [555, 5, 155, 1536.23056636925

In [11]:
newprice = [50,300,500]

In [12]:
client = datastore.Client()
key = client.key('prices', 5631671361601536)
entity = datastore.Entity(key=key)
entity.update({
    "smolprice": newprice[0],
    "chonkprice": newprice[1],
   "absprice": newprice[2]    })
client.put(entity)

## import matplotlib.pyplot as plt

In [111]:
(8-2)/3

2.0

In [112]:
2.0*3+2

8.0